# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451881407 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/mwcruz/anaconda3/envs/dato-env/lib/python2.7/site-packages/certifi/cacert.pem
1451881407 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to m@navegawireless.com and will expire on November 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5178 - Server binary: /home/mwcruz/anaconda3/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451881407.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [164]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.276563     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.300926     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [165]:
non_zero_features = model_all.coefficients[model_all.coefficients['value'] !=0]['name']
print non_zero_features, len(non_zero_features)
model_all.coefficients['value'].nnz()

['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ] 6


6

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [14]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [157]:
import numpy as np

In [168]:
RSS_list=[]
non_zeros=[]
for l1_penalty in np.logspace(1, 7, num=13):
    model_many = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    predictions = model_many.predict(validation)
    errors = predictions - validation['price']
    rss_current = sum(errors**2)
    key = "l1_%.2e" % l1_penalty
    RSS_list.append([rss_current, key])
    non_zeros.append([model_many.coefficients['value'].nnz(), key])
print RSS_list
print non_zeros


    

[[625766285142461.2, 'l1_1.00e+01'], [625766285362395.2, 'l1_3.16e+01'], [625766286057887.0, 'l1_1.00e+02'], [625766288257224.9, 'l1_3.16e+02'], [625766295212186.0, 'l1_1.00e+03'], [625766317206077.8, 'l1_3.16e+03'], [625766386760661.5, 'l1_1.00e+04'], [625766606749281.4, 'l1_3.16e+04'], [625767302791633.4, 'l1_1.00e+05'], [625769507643885.1, 'l1_3.16e+05'], [625776517727025.8, 'l1_1.00e+06'], [625799062845466.9, 'l1_3.16e+06'], [625883719085424.5, 'l1_1.00e+07']]
[[18, 'l1_1.00e+01'], [18, 'l1_3.16e+01'], [18, 'l1_1.00e+02'], [18, 'l1_3.16e+02'], [18, 'l1_1.00e+03'], [18, 'l1_3.16e+03'], [18, 'l1_1.00e+04'], [18, 'l1_3.16e+04'], [18, 'l1_1.00e+05'], [18, 'l1_3.16e+05'], [18, 'l1_1.00e+06'], [18, 'l1_3.16e+06'], [18, 'l1_1.00e+07']]


In [169]:
res = np.array(RSS_list)
best_l1 = np.sort(res)[0]
print "Best l1_penalty is %s, with a RSS value on Validation set of %s" % (best_l1[1], best_l1[0])


Best l1_penalty is l1_1.00e+01, with a RSS value on Validation set of 6.25766285142e+14


In [106]:
small_rss = 1e17
best_l1=0
for pair in RSS_list:
    if pair < small_rss:
        small_rss = pair
    else:
        small_rss =small_rss
print small_rss

6.25766285142e+14


In [173]:
model_best_l1 =  graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1.00e+01, max_iterations=1000)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.272865     | 6306018.232993     | 313413.344721 |
PROGRESS: | 2         | 3        | 0.000002  | 0.295606     | 6019513.777487     | 293369.342637 |
PROGRESS: | 3         | 4      

In [174]:
best_l1_weights = model.coefficients['value']
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)


In [175]:
test_feature_matrix,test_output_array = get_numpy_data(testing, all_features, 'price')
test_prediction = np.dot(test_feature_matrix, best_l1_weights)
test_RSS = sum((test_prediction - testing['price'])**2)
test_RSS

156972779668688.69

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`? = 1e+01
2. What is the RSS on TEST data of the model with the best `l1_penalty`?  156972779668688.69

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [ ]:
18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [176]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [177]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [178]:
non_zeros_list=[]
for l1_penalty in l1_penalty_values:
    key 
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    non_zeros_list.append([model.coefficients['value'].nnz(), l1_penalty])
    

In [193]:
lower_l1_penalty = np.array([ penalty[1] for penalty in non_zeros_list if penalty[0] > max_nonzeros]).max()
upper_l1_penalty = np.array([ penalty[1] for penalty in non_zeros_list if penalty[0] < max_nonzeros]).min()

lower_l1_penalty, upper_l1_penalty


(2976351441.6313133, 3792690190.7322536)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [198]:
l1_penalty_min = lower_l1_penalty
l1_penalty_max = upper_l1_penalty
l1_penalty_min, l1_penalty_max

(2976351441.6313133, 3792690190.7322536)

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 
2976351441.6313133, 3792690190.7322536

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [199]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [213]:
RSS_list_7=[]
non_zeros_7=[]
for l1_penalty in l1_penalty_values:
    model_7 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    predictions_7 = model_7.predict(validation)
    errors_7 = predictions_7 - validation['price']
    rss_current_7= sum(errors_7**2)
    RSS_list_7.append([model_7.coefficients['value'].nnz(), rss_current_7, l1_penalty])
    non_zeros_7.append([model_7.coefficients['value'].nnz(), l1_penalty])
print RSS_list_7
print non_zeros_7

[[10, 966925692362086.8, 2976351441.6313133], [10, 974019450084557.2, 3019316638.9524155], [10, 981188367942450.9, 3062281836.2735176], [10, 989328342459472.0, 3105247033.5946198], [10, 998783211265885.9, 3148212230.9157219], [10, 1008477167020096.5, 3191177428.236824], [10, 1018298780553818.6, 3234142625.5579262], [10, 1028247992205976.1, 3277107822.8790283], [8, 1034616909232829.2, 3320073020.2001305], [8, 1038554735941039.9, 3363038217.5212326], [8, 1043237237871704.1, 3406003414.8423347], [7, 1046937488751713.6, 3448968612.1634369], [7, 1051147625612863.0, 3491933809.484539], [7, 1055992735342999.2, 3534899006.8056412], [7, 1060799531763290.2, 3577864204.1267428], [6, 1065707689498229.6, 3620829401.447845], [6, 1069464335425583.9, 3663794598.7689471], [6, 1073504549585594.6, 3706759796.0900493], [6, 1077632775581412.0, 3749724993.4111514], [6, 1081867592324111.9, 3792690190.7322536]]
[[10, 2976351441.6313133], [10, 3019316638.9524155], [10, 3062281836.2735176], [10, 3105247033.5946

In [214]:
right_sparsity = [penalty for penalty in RSS_list_7 if penalty[0] == max_nonzeros]
right_sparsity

[[7, 1046937488751713.6, 3448968612.1634369],
 [7, 1051147625612863.0, 3491933809.484539],
 [7, 1055992735342999.2, 3534899006.8056412],
 [7, 1060799531763290.2, 3577864204.1267428]]

In [245]:
sparse_model_7 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.1634369, verbose=False)
features_7 = sparse_model_7.coefficients[sparse_model_7.coefficients['value']!=0]
sparse_model_7.coefficients[sparse_model_7.coefficients['value'] != 0].print_rows(num_rows=7)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
+------------------+-------+---------------+
[7 rows x 3 columns]



***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [238]:
print "The l1_penalty with lowest RSS is the %e : " % right_sparsity[0][2]

The l1_penalty with lowest RSS is the 3.448969e+09 : 


In [248]:
print "Features with non_zero coefficients are %s" % features_7['name'][0:]

Features with non_zero coefficients are ['(intercept)', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']
